In [ ]:
# default_exp followers

In [ ]:
%%capture
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"
from nbdev import *
from nbdev.showdoc import *
from fastcore.test import *

/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


# Instagram followers
> Here I'll create a script to fetch new followers for an instagram account. The reason is to track followers change by country, facebook API offers only lifetime numbers which are updated daily.

https://developers.facebook.com/tools/explorer/

https://developers.facebook.com/docs/facebook-login/access-tokens/

## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread facebook_business python-facebook-api

     |████████████████████████████████| 153kB 2.8MB/s 
ERROR: responses 0.11.0 has requirement urllib3>=1.25.10, but you'll have urllib3 1.24.3 which is incompatible.
ERROR: python-facebook-api 0.7.1 has requirement requests<3.0.0,>=2.24.0, but you'll have requests 2.23.0 which is incompatible.


In [ ]:
%%capture
#hide

import yaml
with open("facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
import os
from datetime import datetime, timedelta
from typing import *

import pandas as pd
import numpy as np
from pyfacebook import IgProApi
from pyfacebook.error import *
from tomorrowswig_followers.core import *

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Get new followers with graph api

In [ ]:
#hide
!curl -i -X GET \
 "https://graph.facebook.com/v7.0/17841409206135869/insights?metric=audience_country&period=lifetime&access_token={TOKEN}"

HTTP/2 200 
vary: Accept-Encoding
etag: "ed3fa0dca0ff433c1dbd6f1bfc6621ec8e240b53"
x-business-use-case-usage: {"17841409206135869":[{"type":"instagram","call_count":1,"total_cputime":1,"total_time":1,"estimated_time_to_regain_access":0}]}
content-type: application/json; charset=UTF-8
facebook-api-version: v7.0
strict-transport-security: max-age=15552000; preload
pragma: no-cache
x-fb-rev: 1002668535
access-control-allow-origin: *
cache-control: private, no-cache, no-store, must-revalidate
x-fb-trace-id: DALPc9/NwHZ
x-fb-request-id: Ag5tNqmKACAMaUuQANlDx0l
expires: Sat, 01 Jan 2000 00:00:00 GMT
x-fb-debug: zMYlAgs9lpCC78iSJ6hIcWsfY5OCY6Cxbys6Sr71b3yEJGxcrrtFlm5Donc6rfV9CQxuGJeCKyPQpViOK1g3cA==
date: Tue, 15 Sep 2020 22:03:24 GMT
alt-svc: h3-29=":443"; ma=3600,h3-27=":443"; ma=3600

{"data":[{"name":"audience_country","period":"lifetime","values":[{"value":{"RS":57,"DE":1060,"PR":36,"BD":53,"TW":123,"PT":157,"JO":39,"JP":75,"UA":40,"FR":54,"HU":99,"BR":2609,"MA":83,"SG":763,"DZ":103,"KE"

In [ ]:
api = IgProApi(app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN,
               version="5.0")

In [ ]:
try:
    response = api.get_user_insights(user_id=USER_ID, period="lifetime", metrics=["audience_country"],
                                    return_json=True)
except PyFacebookException as e:
    display(e.message)

In [ ]:
response = response[0]["values"][0]

In [ ]:
response["end_time"]

'2020-09-15T07:00:00+0000'

In [ ]:
#export
def get_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
            user_id=USER_ID,
            period="lifetime",
            metrics=["audience_country"],
            return_json=True,
        )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, {0: 0})

In [ ]:
#export
def get_new_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
                user_id=USER_ID,
                period="day",
                metrics=["follower_count"],
                return_json=True,
            )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, 0)

In [ ]:
get_new_followers()

('2020-09-14', 2)

## Google Sheets

### Insert new followers to a google sheet 

In [ ]:
end_time, followers = get_followers()

In [ ]:
df = get_df("History")

In [ ]:
new_followers = pd.Series(followers)
date = (datetime.strptime(end_time, "%Y-%m-%d") - timedelta(days=1)).strftime(f"%b %d %Y{' '*16}")
new_followers.name = f"{date} {str(datetime.utcnow()).split('.')[0]}"
last_entry = df.iloc[:, 0]

In [ ]:
if np.array_equal(last_entry[last_entry != 0].values, new_followers.sort_index().values):
    # return pd.DataFrame()
    pass
else:
    df = pd.concat([df, new_followers], axis=1)
    df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
    df.index.name = "countries" 
    df.sort_index(inplace=True)

In [ ]:
df = df.fillna(0).astype(int)

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df


def get_followers_change(history_df: pd.DataFrame) -> pd.DataFrame:
    new_followers = get_dif(history_df)
    new_followers = new_followers.iloc[:,[0]].replace(0, np.nan)
    new_followers = new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)
    return new_followers

In [ ]:
#export
def get_ads_status(date: str):
    if get_df(date).empty:
        return "OFF"
    return "ON"

In [ ]:
history_df = get_df("History")

In [ ]:
ads_status_df = get_df("Ads Status History")
date = history_df.columns[0].split("  ")[0]
ads_status = get_ads_status(date)
change = get_followers_change(history_df).sum().item()

In [ ]:
last_status = pd.Series([ads_status, change, ""], name="", dtype=str, index=["Ads Status", "Change", "Followers"])
last_status.name = date
last_status

Ads Status    OFF
Change        -42
Followers        
Name: Sep 14 2020, dtype: object

In [ ]:
ads_status_df

,Sep 13 2020
,
Ads Status,OFF
Change,-30


In [ ]:
pd.concat([last_status, ads_status_df], axis=1)

,Sep 14 2020,Sep 13 2020
Ads Status,OFF,OFF
Change,-42,-30
Followers,,NaN


In [ ]:
#export
def save_ads_status(history_df: pd.DataFrame):
    worksheet = "Ads Status History"
    ads_status_df = get_df(worksheet)
    date = history_df.columns[0].split("  ")[0]
    ads_status = get_ads_status(date)
    change = get_followers_change(history_df).sum().item()
    last_status = pd.Series([ads_status, change], name="", dtype=str, index=["Ads Status", "Change"])
    last_status.name = date
    write_df(pd.concat([last_status, ads_status_df], axis=1), worksheet)

In [ ]:
# save_ads_status(get_df("History"))

In [ ]:
#export
def get_updated_followers(
    df: pd.DataFrame, data: Dict[str, int], end_time: str
) -> pd.DataFrame:
    new_followers = pd.Series(data)
    date = (datetime.strptime(end_time, "%Y-%m-%d") - timedelta(days=1)).strftime(f"%b %d %Y{' '*16}")
    new_followers.name = f"{date} {str(datetime.utcnow()).split('.')[0]}"
    last_entry = df.iloc[:, 0]
    second_last_entry = df.iloc[:, 1]
    if np.array_equal(last_entry[last_entry != 0].values, new_followers.sort_index().values) or np.array_equal(second_last_entry[second_last_entry != 0].values, new_followers.sort_index().values):
        return pd.DataFrame()
    else:
        df = pd.concat([df, new_followers], axis=1)
        df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
        df.index.name = "countries" 
        df.sort_index(inplace=True)
        return df.fillna(0).astype(int)

In [ ]:
get_updated_followers(history_df, get_followers()[1], get_followers()[0])

,Sep 14 2020 2020-09-15 22:03:32,Sep 13 2020 2020-09-14 23:10:22,Sep 13 2020 2020-09-14 22:20:24,Sep 13 2020 2020-09-14 21:00:30,Sep 12 2020 2020-09-13 23:40:39,Sep 11 2020 2020-09-13 20:56:38,Aug 31 2020 2020-09-01 21:40:38,Aug 30 2020 2020-08-31 20:20:31,Aug 29 2020 2020-08-30 20:11:59,Aug 28 2020 2020-08-29 18:10:38,Aug 27 2020 2020-08-28 22:00:22,Aug 26 2020 2020-08-28 03:10:23,Aug 26 2020 2020-08-28 02:40:30,Aug 26 2020 2020-08-28 02:10:31,Aug 25 2020 2020-08-27 00:30:30,Aug 24 2020 2020-08-25 19:20:23,Aug 23 2020 2020-08-25 01:20:22,Aug 22 2020 2020-08-23 21:50:22,Aug 21 2020 2020-08-22 23:00:22,Aug 20 2020 2020-08-21 20:40:25,Aug 19 2020 2020-08-21 06:20:22,Aug 18 2020 2020-08-20 04:20:30,Aug 17 2020 2020-08-18 19:20:38,Aug 16 2020 2020-08-17 21:00:23,Aug 15 2020 2020-08-16 20:50:25,Aug 14 2020 2020-08-15 20:20:22,Aug 13 2020 2020-08-14 19:40:33,Aug 12 2020 2020-08-13 20:20:22,Aug 11 2020 2020-08-13 02:16:27,Aug 10 2020 2020-08-12 02:51:46,Aug 09 2020 2020-08-10 19:52:01,Aug 08 2020 2020-08-09 20:12:05,Aug 07 2020 2020-08-08 18:47:00,Aug 06 2020 2020-08-07 22:26:58,Aug 05 2020 2020-08-07 02:10:42,Aug 04 2020 2020-08-06 02:10:39,Aug 03 2020 2020-08-05 02:10:40,Aug 02 2020 2020-08-04 01:10:37,Aug 01 2020 2020-08-03 01:10:56,Jul 31 2020 2020-08-02 01:10:45,...,Aug 28,Aug 27,Aug 26,Aug 25,Aug 20,Aug 19,Aug 18,Aug 17,Aug 16,Aug 15,Aug 14,Aug 13,Aug 12,Aug 11,Aug 10,Aug 9,Aug 8,Aug 7,Aug 6,Aug 5,Aug 4,Aug 3,Aug 2,Aug 1,Jul 31,Jul 30,Jul 29,Jul 28,Jul 27,Jul 25,Jul 24,Jul 23,Jul 22,Jul 21,Jul 18,Jul 17,Jul 16,Jul 14,Jun 23,Jun 21
countries,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AR,1011,1012,1015,1012,1015,1018,1031,1033,1034,1033,1029,1030,1031,1030,1031,1031,1032,1032,1033,1035,1034,1035,1037,1037,1034,1033,1038,1039,1042,1046,1047,1047,1050,1052,1055,1054,1056,1058,1060,1062,...,233,233,234,235,237,237,237,237,237,239,238,239,238,238,238,239,239,239,239,240,241,241,241,241,242,242,244,244,244,244,246,245,245,245,250,250,250,250,218,145
AU,934,935,933,935,933,934,944,944,934,925,917,917,919,917,919,921,921,909,887,886,886,887,885,887,885,866,874,876,878,879,881,873,861,854,852,853,853,853,844,828,...,66,68,67,68,66,66,66,66,66,66,66,64,63,63,63,63,60,53,51,50,51,51,45,42,0,42,42,0,0,0,0,0,0,0,0,0,0,0,40,41
BD,53,53,53,53,53,54,55,55,55,55,55,55,55,55,55,55,54,55,55,55,55,55,55,55,55,55,55,54,53,53,53,54,54,54,54,54,54,54,54,55,...,79,78,80,80,80,80,80,80,80,79,79,78,79,79,79,79,79,79,79,79,80,80,80,81,81,81,81,81,82,82,81,81,81,81,83,83,83,83,88,84
BR,2609,2613,2612,2613,2612,2618,2641,2641,2595,2553,2545,2546,2550,2546,2550,2550,2556,2522,2497,2496,2501,2500,2508,2511,2469,2451,2447,2451,2454,2460,2467,2423,2384,2380,2387,2393,2394,2394,2356,2316,...,1214,1200,1181,1156,1081,1071,1060,1046,1033,1011,987,971,953,951,923,904,897,888,873,863,852,831,818,810,791,775,760,742,731,684,669,654,644,627,595,566,547,504,248,164
CA,1873,1876,1874,1876,1874,1873,1896,1899,1859,1864,1865,1872,1877,1872,1877,1875,1875,1834,1834,1836,1839,1841,1847,1849,1807,1815,1826,1823,1825,1826,1826,1780,1782,1782,1782,1782,1784,1787,1732,1735,...,156,157,158,157,161,161,159,155,150,148,141,139,132,129,126,125,119,116,113,109,105,104,104,102,98,94,91,90,86,81,81,77,76,74,69,66,63,59,56,35
CL,501,502,502,502,502,504,512,513,513,513,512,511,511,511,511,512,513,514,515,515,514,515,517,517,518,518,519,519,521,520,521,521,520,520,520,519,521,520,520,520,...,212,214,215,215,217,216,215,216,216,217,217,216,216,218,219,219,219,219,219,219,219,220,220,220,220,222,222,222,222,225,226,226,226,226,226,228,227,226,229,177
CO,62,62,62,62,62,62,63,63,61,61,61,61,61,61,61,61,60,61,61,61,60,60,61,61,62,62,62,62,62,62,62,62,61,61,60,60,60,61,61,61,...,65,66,66,66,65,65,65,65,65,65,65,65,65,66,67,67,67,67,67,67,67,67,67,67,67,67,67,67,66,66,66,66,66,67,67,67,67,67,68,62
CR,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,70,71,71,71,70,70,70,70,70,70,70,70,70,70,70,70,70,70,69,69,69,...,85,85,85,85,85,85,85,85,85,84,84,85

In [ ]:
test_followers_df.to_dict()

{'2020-01-01': {'AR': 1001,
  'AU': 724,
  'BD': 55,
  'BR': 2250,
  'CA': 1680,
  'CL': 526,
  'CO': 6},
 '2020-07-20': {'AR': 100,
  'AU': 70,
  'BD': 0,
  'BR': 0,
  'CA': 0,
  'CL': 0,
  'CO': 1},
 '2020-07-21': {'AR': 100,
  'AU': 70,
  'BD': 0,
  'BR': 0,
  'CA': 0,
  'CL': 0,
  'CO': 0}}

In [ ]:
fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"
old_followers = {end_time: {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 0},
                 "2020-07-20": {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 1}}
followers_df = pd.DataFrame.from_records(old_followers)

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)

test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
expected = {'2020-01-01': {'AR': 1001,
                            'AU': 724,
                            'BD': 55,
                            'BR': 2250,
                            'CA': 1680,
                            'CL': 526,
                            'CO': 6},
 '2020-07-20': {'AR': 100,
                'AU': 70,
                'BD': 0,
                'BR': 0,
                'CA': 0,
                'CL': 0,
                'CO': 1},
 '2020-07-21': {'AR': 100,
                'AU': 70,
                'BD': 0,
                'BR': 0,
                'CA': 0,
                'CL': 0,
                'CO': 0}}
assert test_followers_df.to_dict() == expected

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
assert test_followers_df.to_dict() == expected

In [ ]:
# test the fix for when API returns an old result shortly after it returned an updated one

fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"
old_followers = {"2020-07-21": {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 0},
                 "2020-07-20": {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}}
followers_df = pd.DataFrame.from_records(old_followers)

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)
assert test_followers_df.empty

In [ ]:
#export
def add_followers(df: pd.DataFrame, new_followers: pd.DataFrame):
    df["Ads Followers Change"] = new_followers
    df.fillna(0, inplace=True)
    df["% (Clicks)"] = (df["Ads Followers Change"] / df["Clicks (All)"])
    df["% (Impressions)"] = (df["Ads Followers Change"] / df["Impressions"])
    df["Cost Per Follow"] = (df["Amount Spent (USD)"] / df["Ads Followers Change"])
    df.replace([np.inf], 0, inplace=True)
    df["% (Impressions)"] = df["% (Impressions)"].round(4)
    df[df.columns.difference(["% (Impressions)"])] = df[df.columns.difference(["% (Impressions)"])].round(2)

random followers + followers from ads - unfollowers = followers gain

unfollowers = random followers + followers from ads - followers gain = new followers - followers gain

random_followers = new followers - followers from ads

In [ ]:
#export
def more_stats(df: pd.DataFrame, followers_change: pd.DataFrame) -> pd.DataFrame:
    stats = pd.Series(dtype=float, name="Followers")
    ads_followers_gain = df["Ads Followers Change"].sum()
    followers_gain = followers_change.sum().item()
    spent = df.loc["total", "Amount Spent (USD)"]
    stats["Ads Followers Change"] = ads_followers_gain
    new_followers = get_new_followers()[1]
    stats["Random Followers"] = new_followers - ads_followers_gain
    stats["New Followers"] = new_followers
    stats["Unfollowers"] = stats["New Followers"] - followers_gain
    stats["Followers Change"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

    stats_df = pd.DataFrame(stats)
    stats_df.index.name = "  "
    stats_df["Conversion Rate"] = stats_df["Followers"] / df.loc["total", "Clicks (All)"]
    stats_df[" "] = 0
    stats_df["Cost Per Follower/Increase"] = spent/ stats_df["Followers"]
    stats_df.iloc[1:, 1] = 0
    stats_df.iloc[[1,3,4], 3] = 0
    return stats_df.round(2)

In [ ]:
date = "Aug 13 2020"
insights_df = get_df(date)
history_df = get_df("History")

In [ ]:
followers_change = get_followers_change(history_df)
df = get_df(date)
if "PERFORMANCE:" in df.index:
    df = df.iloc[:df.index.get_loc("PERFORMANCE:") - 9, :]

In [ ]:
add_followers(df, followers_change)

In [ ]:
stats_df = more_stats(df, followers_change)

In [ ]:
get_followers_change(history_df)

,Sep 13 2020 2020-09-14 23:10:22
countries,
AR,-3
AU,2
BR,1
CA,2
CZ,1
DE,-1
DZ,1
GB,-9
GR,1


In [ ]:
#export
def update_insights(history_df: pd.DataFrame, date: str):
    followers_change = get_followers_change(history_df)
    df = get_df(date)
    if df.empty:
        return f"Nothing to update for {date}"
    if "PERFORMANCE:" in df.index:
        df = df.iloc[:df.index.get_loc("PERFORMANCE:") - 9, :]

    add_followers(df, followers_change)
    stats_df = more_stats(df, followers_change)

    empty = pd.Series(name="", dtype=str)
    df = df.append([empty] * 4)
    write_df(df, date)
    write_df(stats_df, date, loc=f"B{len(df)}")
    wsh = get_worksheet(date)
    wsh.format(f"C{wsh.find('Unfollowers').row}", {"textFormat": {
        "foregroundColor": {
            "red": 0.0,
            "green": 0.0,
            "blue": 1.0
        }}},)

In [ ]:
# update_insights(history_df, date)

In [ ]:
#export
def save_followers() -> str:
    df = get_df("History")
    end_time, followers = get_followers()
    df = get_updated_followers(df, followers, end_time)
    if df.empty:
        return f"No followers change, end_time '{end_time}'"
    date = df.columns[0].split("  ")[0]
    write_df(df, "History")
    save_ads_status(df)
    update_insights(df, date)
    return f"Wrote followers and insights for '{date}'"

In [ ]:
save_followers()

"Wrote followers and insights for 'Sep 14 2020'"

### Get difference from the previous day

In [ ]:
df = get_df("History")

In [ ]:
change_df = df.diff(axis=1, periods=-1)

In [ ]:
change_df = change_df.fillna(0).astype(int)

In [ ]:
mask = (change_df != 0).any(axis=0)

In [ ]:
change_df = change_df.loc[:, mask]

In [ ]:
change_df = change_df.applymap(lambda x: f"+{x}" if x >0 else x)

In [ ]:
change_df = "(" + change_df.astype(str) + ")" + " " + df.loc[:, mask].astype(str)

In [ ]:
change_df = change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
#export
def make_change(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    zeros_mask  = (change_df != 0).any(axis=0)
    change_df = change_df.loc[:, zeros_mask]
    change_df = change_df.applymap(lambda x: f"+{x}" if x > 0 else x)
    change_df = "(" + change_df.astype(str) + ")" + " " + df.loc[:, zeros_mask].astype(str)
    return change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
#export
def save_change():
    df = get_df("History")
    change_df = make_change(df)
    write_df(change_df, "Change History")
    ads_status = get_df("Ads Status History")
    write_df(ads_status, "Change History", f"A{len(change_df)+3}", columns=False)

In [ ]:
save_change()

In [ ]:
test_change_df = make_change(pd.DataFrame({"2020-07-21": [1001, 724, 55, 2250, 1680],
                                           "2020-07-20": [1004, 708, 55, 2300, 1600],
                                           "2020-07-19": [1004, 708, 55, 2300, 1600]},
                                          index=["AR", "AU", "BD", "BR", "CA"]))
assert test_change_df.iloc[4,:].to_list() == ["(+80) 1680"]
assert test_change_df.iloc[3,:].to_list() == ["(-50) 2250"]

In [ ]:
#export
def update(event: Dict = None, context=None,):
    get_followers()
    message = save_followers()
    save_change()
    return message

In [ ]:
update()

"No followers change, end_time '2020-09-15'"

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
